实验计划

模型架构mlp+softmax多分类

1.实现pytorch版,并且稍微调参

2.复刻myTorch版

3.基于myTorch做消融实验(优化器，初始化策略)

In [1]:
#把项目目录添加至环境变量
import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()

# 获取上级目录
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
print(parent_dir)
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

c:\Users\nju22\Desktop\myTorch


In [ ]:
# 基于pytorch导入数据
import torch
from torchvision import datasets, transforms
from torch.utils.data import random_split
import MyTorch.Dataloader
data_dir = "../../dataset"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
data_transforms = transforms.Compose([
    transforms.ToTensor(),  # 转换为Tensor
    transforms.Normalize((0.5,), (0.5,)),  # 归一化，均值为0.5，标准差为0.5
])
# MNIST dataset
train_data = datasets.MNIST(data_dir, train=True, download=True, transform=data_transforms)
test_data = datasets.MNIST(data_dir, train=False, download=True, transform=data_transforms)

print(f"shape:{train_data.data.shape}")
print(f"mean:{train_data.data.float().mean()}")#pytorch dataset的transform只会在取数据进入model时调用，故此时print仍然是原始数据

val_size = int(0.1 * len(train_data))
train_data, val_data = random_split(train_data, [len(train_data) - val_size, val_size])
print("train_data:", len(train_data))
print("val_data:", len(val_data))


shape:torch.Size([60000, 28, 28])
mean:33.31842041015625
train_data: 54000
val_data: 6000


In [3]:
#定义固定参数
input_size = 28 * 28
num_classes = 10
#定义超参
batch_size = 64
hidden_size = 128
lr=0.001
epochs = 1000

In [4]:
# 构建model
from torch import nn
model=torch.nn.Sequential(nn.Linear(input_size, hidden_size),nn.ReLU(),nn.Linear(hidden_size, num_classes),nn.Softmax(dim=1))

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
train_loader =torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i + 1) % 100 == 0:
            print('Epcho [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch, epochs, i + 1, len(train_loader), loss.item()))
    

TypeError: Tensor is not a torch image.